In [56]:
import numpy as np
import tensorflow as tf

import pickle
import zipfile
import collections
import math
import time

from collections import Counter

1. Preprocessing the data
===

In [2]:
def preprocess(file_name):
    with zipfile.ZipFile(file_name) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

words = preprocess('text8.zip')

In [3]:
# make word list, dictionary and reverse dictionary
def word_dictionary(words):
    
    ### words : total words list of text8 dataset
    word_lst = [['UNK',0]]
    word_dict = dict()
    
    # total 100000 most common (word, frequency) pair
    word_lst.extend(Counter(words).most_common(99999)) 
    
    for idx, (word, _) in enumerate(word_lst):
        # dictionary : {'word' : 'index'}
        word_dict[word] = idx
        
    # reverse dictionary : {'index' : 'word'}
    word_reverse_dict = dict(zip(word_dict.values(), word_dict.keys()))
    
    return word_lst, word_dict, word_reverse_dict

#word_lst, word_dict, word_reverse_dict = word_dictionary(words)

In [4]:
# 기존 dataset에 존재하는 단어에서 상위 10만개 이외의 단어들을 
def unknown_count(words, word_lst, word_dict):
    words_index = list()
    
    for word in words:
        if word in word_dict:
            idx = word_dict[word]
        else:
            idx = 0
            word_lst[0][1] += 1 # 'UNK' count
            
        words_index.append(idx)
    return words_index, word_lst

#words_index, word_lst = unknown_count(words, word_lst, word_dict)

In [5]:
# Subsampling function
def Subsampling(word_lst, words_index, threshold):
    word_prob = dict()
    
    for idx, (word, freq) in enumerate(word_lst):
        prob = freq / len(words_index) # make probability distribution of word
        word_prob[idx] = 1 - np.sqrt(threshold / prob)
        
    return [idx for idx in words_index if word_prob[idx] < 1 - np.random.random()]

In [6]:
def Unigram_dict(words, word_lst):
    unigram = {}
    
    for idx, (word, freq) in enumerate(word_lst):
        unigram[word] = freq / len(words)
        
    return unigram

In [7]:
def total_sampling (file_data=words, subsampling=False, threshold=None):
    start = time.time()
    
    word_lst, word_dict, word_reverse_dict = word_dictionary(file_data)   # 10만개 (word, freq) pair, word-index dict and reverse dict
    make_dict_time = time.time()
    
    words_index, word_lst = unknown_count(file_data, word_lst, word_dict) # words_index for subsampling, unknown count
    unk_count = time.time()
    
    if subsampling:
        words_index = Subsampling(word_lst, words_index, threshold)
        subsample_time = time.time()
    
    unigram_dict = Unigram_dict(words, word_lst)
    unigram_time = time.time()
    
    print ('making dictionary : ', make_dict_time - start)
    print ('unknown count : ', unk_count - make_dict_time)
    print ('subsampling time : ', subsample_time - unk_count)
    print ('unigram dictionary : ', unigram_time - subsample_time)
    
    return file_data, word_lst, word_dict, word_reverse_dict, words_index, unigram_dict

In [8]:
words, word_lst, word_dict, word_reverse_dict, words_index, unigram_dict = total_sampling(file_data=words, subsampling=True, threshold=1e-5)

making dictionary :  2.460446357727051
unknown count :  3.2403206825256348
subsampling time :  9.886610984802246
unigram dictionary :  0.044852495193481445


2. Make Model in training
=====

In [82]:
def Sigmoid(x):
    # clip input to -10~10
    x = np.clip(x, -10, 10)
    return (1 / 1 + np.exp(-x))

In [83]:
def word2vec_test():
    dummy_vectors = np.random.randn(100000, 300)
    Skipgram('the', ['the', 'anarchism', 'has', 'gone'], inputVectors=dummy_vectors[:50000,:],
            outputVectors = dummy_vectors[50000:,:], use_loss='Softmax', tokens=word_dict)

    CBOW('the', ['the', 'anarchism', 'has', 'gone'], inputVectors=dummy_vectors[:50000,:],
            outputVectors = dummy_vectors[50000:,:], use_loss='Softmax', tokens=word_dict)

In [80]:
def Skipgram(currentWord, contextWords, inputVectors, outputVectors,
             use_loss='Softmax', tokens=None, negative_sampling = None):
    """
    currentWord    : center word String
    contextWords   : list of string, which has 2 * C words
    inputVectors   : access by row number, such as inputVectors[5]...
    outputVectors  : access by row number same as inputVectors
    use_loss       : Softmax, Hierarchical Softmax, etc...
    tokens         : get index used at inputVectors
    """
    loss = 0.0
    N, V = inputVectors.shape
    C = len(contextWords)
    
    gradsIn = np.zeros(inputVectors.shape)
    gradsOut = np.zeros(outputVectors.shape)
    hiddenVectors = np.zeros((1, V))
    score = np.zeros((N, 1))
    
    index = tokens[currentWord]
    hiddenVectors = inputVectors[index] # current word to hidden vector
    
    if use_loss == 'Softmax':
        # Softmax forwarding 
        for i in range(C):
            word = contextWords[i]
            word_idx = tokens[word]
            score[index] += np.dot(hiddenVectors, outputVectors[index].T) # (1, 1)
        
        exp_out = np.exp(score)
        softmax = exp_out / np.sum(exp_out)
        dsoftmax = softmax.copy()
        # Softmax backwarding
        for i in range(C):
            word = contextWords[i]
            word_idx = tokens[word]
            loss = -np.sum(np.log(softmax[word_idx]))
            dsoftmax[word_idx] -= 1
            
        loss /= N
        #print (dsoftmax.shape, hiddenVectors.shape)
        gradsOut = np.dot(dsoftmax, hiddenVectors.reshape(1,-1))
        gradshidden = np.dot(dsoftmax.T, outputVectors)
        gradsIn[index] = gradshidden
        
        # return loss, gradsIn, gradsOut
        
    #elif use_loss == 'Hierarchical_Softmax':
        
    elif use_loss == 'Negative_Sampling':
        # get unigram distribution and negative sampling index
        negative_sample_idx = negative_sampling.sampling(index)
        # negative sampling model forwarding
    return loss, gradsIn, gradsOut

In [81]:
def CBOW(currentWord, contextWords, inputVectors, outputVectors,
             use_loss='Softmax', tokens=None, negative_sampling=None):
    """
    currentWord    : center word String
    contextWords   : list of string, which has 2 * C words
    inputVectors   : access by row number, such as inputVectors[5]...
    outputVectors  : access by row number same as inputVectors
    use_loss       : Softmax, Hierarchical Softmax, etc...
    tokens         : get index used at inputVectors
    """
    loss = 0.0
    N, V = inputVectors.shape
    C = len(contextWords)
    
    gradsIn = np.zeros(inputVectors.shape)
    gradsOut = np.zeros(outputVectors.shape)
    hiddenVectors = np.zeros((1, V))
    score = np.zeros((N, 1))
    
    index = tokens[currentWord]
    
    if use_loss == 'Softmax':
        # Softmax forwarding
        for i in range(C):
            word = contextWords[i]
            word_idx = tokens[word]
            hiddenVectors += inputVectors[word_idx]
        
        score = np.dot(outputVectors, hiddenVectors.T)
        exp_score = np.exp(score)
        softmax = exp_score / np.sum(exp_score)
        dsoftmax = softmax.copy()
        
        # Softmax backwarding
        loss = -np.sum(np.log(softmax[index]))
        loss /= N
        
        dsoftmax[index] -= 1
        gradsOut = np.dot(dsoftmax, hiddenVectors)
        gradshidden = np.dot(dsoftmax.T, outputVectors)
        
        for i in range(C):
            word = contextWords[i]
            word_idx = tokens[word]
            gradsIn[word_idx] = gradshidden

        # return loss, gradsIn, gradsOut
        
    #elif use_loss == 'Hierarchical_Softmax':
        
    elif use_loss == 'Negative_Sampling':
        # get unigram distribution and negative sampling index
        negative_sample_idx = negative_sampling.sampling(index)
        # negative sampling model forwarding
        
    return loss, gradsIn, gradsOut

## Make Negative Sampling model

In [48]:
class Unigram_Table:
    #--- 3/4 power of unigram distribution selected by mikolov et al. 2013
    #--- make table and recall with index
    """
    A list of indices of tokens in the vocab following a power law distribution,
    used to draw negative samples.
    """
    def __init__(self, vocab, count, unigram_dictionary=dict()):
        self.count = count
        vocab_size = len(vocab)
        power = 0.75
        table_size = int(1e8) # unigram table length
        table = np.zeros(table_size, np.uint32)
        
        norm_val = sum(math.pow(uni_prob, power) for uni_prob in unigram_dictionary.values())
        
        print ('Filling Unigram Table')
        p = 0 # Cumulative Probability
        i = 0
        
        for idx, (word, prob) in enumerate(unigram_dictionary.items()):
            p += float(math.pow(prob, power)) / norm_val
            while (i < table_size) and (float(i) / table_size < p):
                table[i] = idx
                i += 1
                
        self.table = table
        
    def sampling(self, true_idx):
        """
        true_idx : index of target word
        return : except target word, count number of whole words
        """
        while True:
            indices = np.random.randint(low=0, high=len(self.table), size=self.count)
            if true_idx not in indices:
                break
            
        return [self.table[i] for i in indices]
    
table = Unigram_Table(words, count=5, unigram_dictionary=unigram_dict)

Filling Unigram Table


In [ ]:
def low_freq_pair(freq_dict):
    """
    freq_dict : (word, word_frequency) pair of dictionary
    """
    low_prob_word = sorted(freq_dict.items(), key=lambda x : x[1])
    
    return low_prob_word[0][0], low_prob_word[1][0]

In [ ]:
### huffman code ###
def huffman_coding(freq_dict):
    """
    freq_dict : (word, word_frequency) pair of dictionary
    """
    
    # when 2 word left, it gives the 0 and 1 binary equally
    if (len(freq_dict) == 2):
        return dict(zip(freq_dict.keys(), ['0', '1']))
    
    # making all frequency as tree
    new_freq_dict = freq_dict.copy()
    word_1, word_2 = low_freq_pair(freq_dict) # 2 word which has low frequency
    freq_1, freq_2 = new_freq_dict.pop(word_1), new_freq_dict.pop(word_2) # frequency of each word
    new_freq_dict[word_1 + word_2] = freq_1 + freq_2
    
    # recursive function
    new_dict = huffman_coding(new_freq_dict)
    add_freq = new_dict.pop(word_1 + word_2)
    new_dict[word_1], new_dict[word_2] = add_freq + '0', add_freq + '1'
    
    return new_dict